In [2]:
!pip install pillow

In [2]:
import cv2
import tensorflow as tf
import numpy as np
from numpy import asarray

/Users/hotpolarbear/Documents/Programs/Projects/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
image = cv2.imread("/Users/hotpolarbear/Documents/Programs/Projects/OCR/Segmentation/Sample_Image_02.png")

In [4]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (3,3), 0)


thresh = cv2.threshold(blur, 100,255 ,cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 10))
dilate = cv2.dilate(thresh, kernel , iterations = 1)


cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts =cnts[0] if len(cnts) == 2 else cnts[1]
cnts = sorted(cnts, key = lambda x : cv2.boundingRect(x)[0])

In [5]:
label_list=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

In [6]:
model=tf.keras.models.load_model("/Users/hotpolarbear/Documents/Programs/Projects/OCR/Training/OCR_N_25M_9181.keras")

In [7]:
ans =""

for c in cnts:
    x,y,w,h = cv2.boundingRect(c)
    if h>20 and w >2 :
        roi = image[y:y+h, x:x+w]
        cv2.rectangle(image, (x,y), (x+w, y+h), (36,255,12), 2)
        cv2.imshow("Image", roi); cv2.waitKey(0); cv2.destroyAllWindows(); cv2.waitKey(1)
        img=cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        _, img = cv2.threshold(img, 100, 255, cv2.THRESH_BINARY)


        img = cv2.resize(img, (128, 128))
        #img = tf.convert_to_tensor(img)
        img=img/255.0
        img = np.expand_dims(img, axis=0)
        '''x = asarray(img)
        x = x.astype('float32')/255.0

        x = np.expand_dims(x, axis=0)
        
        x = np.where(x=='True', 1, x)
        x = np.where(x=='False', 0, x)
        
        print(type(x))'''
        print(img.shape)


        ocr_result = model.predict(img)

        print("highest accuracy " + str(ocr_result.max()))
        max = ocr_result.max()
        index = ocr_result.argmax()

        ans = ans + str((label_list[index]))


2025-01-22 13:33:15.158 Python[52539:12913823] +[IMKClient subclass]: chose IMKClient_Modern
2025-01-22 13:33:15.158 Python[52539:12913823] +[IMKInputSession subclass]: chose IMKInputSession_Modern


(1, 128, 128)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
highest accuracy 0.99999523
(1, 128, 128)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
highest accuracy 0.39235577
(1, 128, 128)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
highest accuracy 0.9353859
(1, 128, 128)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
highest accuracy 0.9353859
(1, 128, 128)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
highest accuracy 0.6974366


In [8]:
ans

'H9IIO'